<a href="https://colab.research.google.com/github/Matheus-lucas/Grokking_Deep_learning/blob/master/8_Learning_Signal_and_Ignoring_Noise_Introduction_to_Regularization_%26_Batching_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pip install tensorflow-gpu

In [ ]:
import numpy as np

# digits images dataset
from keras.datasets import mnist  

import time

Using TensorFlow backend.


# Training with MNIST Dataset

In [ ]:
%%time
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# reshaping the matrix and normalizing the pixels colors (reminder than 0==zero and 255==white) dividing by 255
images, labels = (x_train[0:1000].reshape(1000,28*28)/255, y_train[0:1000])  

# create a matrix of zeros
one_hot_labels = np.zeros((len(labels),10))  

# make one_hot_label[i][l], where i(index of labels values) and l(labels values)), be 1 
for i,l in enumerate(labels):  
    one_hot_labels[i][l] = 1
    
labels = one_hot_labels  

# create the tests arrays
test_images = x_test.reshape(len(x_test),28*28) / 255  
test_labels = np.zeros((len(y_test),10))

# make test_labels[i][l], where i(index of y_test values) and l(y_test values)), be 1 
for i,l in enumerate( y_test):  
    test_labels[i][l] = 1
    
np.random.seed(1)  
relu = lambda x:(x>=0) * x # returns x if x > 0, return 0 otherwise  

relu2deriv = lambda x: x>=0 # returns 1 for input > 0, return 0 otherwise  

alpha, iterations, hidden_size, pixels_per_image, num_labels = \
    (0.005, 350, 40, 784, 10)  

weights_0_1 = 0.2*np.random.random((pixels_per_image,hidden_size)) - 0.1  

weights_1_2 = 0.2*np.random.random((hidden_size,num_labels)) - 0.1  
corr,error2=0.0,0

time.sleep(5)
for j in range(iterations):
    error, correct_cnt = 0.0, 0
    
    for i in range(len(images)):  
        
        layer_0 = images[i:i+1]  
        layer_1 = relu(np.dot(layer_0,weights_0_1))  
        layer_2 = np.dot(layer_1,weights_1_2)  
        
        # calculate the error between labels and layer, with each number at a time
        error += np.sum((labels[i:i+1] - layer_2) ** 2) 

        # counter the correct answer, comparing with the labels arg max (1)
        # reminder than layer and labels are arrays in format [0, 0, 1, 0] and [0.34, .65, .67, 1] and picking the argmax is pick the index of 1, and compare
        correct_cnt += int(np.argmax(layer_2) == \
                           np.argmax(labels[i:i+1]))  
       
        # calculate deltas
        layer_2_delta = (labels[i:i+1] - layer_2) 
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) \
            * relu2deriv(layer_1)  
        
        # upgrade the weights with gradiente descendent
        weights_1_2 += alpha * layer_1.T.dot(layer_2_delta) 
        weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)  
        
        # calculate the mean of errors
        error2 = error / float(len(images))

        # calculate the % of corrects answer
        corr = correct_cnt / float(len(images))
        if ((j % 10 == 0 and i%10==0)or j == iterations- 1): 
            print("I: {} Error: {:.5f} Correct: {:.5f}".format(j , error2,corr))

I: 0 Error: 0.00093 Correct: 0.00000
I: 0 Error: 0.01408 Correct: 0.00000
I: 0 Error: 0.02576 Correct: 0.00100
I: 0 Error: 0.03726 Correct: 0.00200
I: 0 Error: 0.04667 Correct: 0.00300
I: 0 Error: 0.05636 Correct: 0.00300
I: 0 Error: 0.06651 Correct: 0.00400
I: 0 Error: 0.07512 Correct: 0.00700
I: 0 Error: 0.08471 Correct: 0.00900
I: 0 Error: 0.09395 Correct: 0.01100
I: 0 Error: 0.10280 Correct: 0.01500
I: 0 Error: 0.11271 Correct: 0.01700
I: 0 Error: 0.12070 Correct: 0.01900
I: 0 Error: 0.12874 Correct: 0.02300
I: 0 Error: 0.13826 Correct: 0.02300
I: 0 Error: 0.14740 Correct: 0.02400
I: 0 Error: 0.15568 Correct: 0.02800
I: 0 Error: 0.16347 Correct: 0.03200
I: 0 Error: 0.17137 Correct: 0.03500
I: 0 Error: 0.17938 Correct: 0.04000
I: 0 Error: 0.18833 Correct: 0.04200
I: 0 Error: 0.19586 Correct: 0.04500
I: 0 Error: 0.20388 Correct: 0.05100
I: 0 Error: 0.21207 Correct: 0.05500
I: 0 Error: 0.22023 Correct: 0.05800
I: 0 Error: 0.22872 Correct: 0.06100
I: 0 Error: 0.23739 Correct: 0.06500
I

### Testing

In [ ]:
%%time
for j in range(iterations):

  if (j % 10 == 0 or j == iterations- 1):  
    error, correct_cnt = 0.0, 0

    for i in range(len(test_images)):  
      layer_0 = test_images[i:i+1]  
      layer_1 = relu(np.dot(layer_0,weights_0_1))  
      layer_2 = np.dot(layer_1,weights_1_2)

      # calculate the errors between the vectors
      error += np.sum((test_labels[i:i+1] - layer_2) ** 2)  

      # counter the correct answer, comparing with the labels arg max (1)
      # reminder than layer and labels are arrays in format [0, 0, 1, 0] and [0.34, .65, .67, 1], so picking the argmax is pick the index of 1, and compare
      correct_cnt += int(np.argmax(layer_2)  == np.argmax(test_labels[i:i+1]))  

      # calculate the mean error
      error2 = error / float(len(test_images))

      # dividing by len(test_images) for pcik the % of correct answer
      corr = correct_cnt / float(len(test_images))

print("Error: {:.5f} Correct: {:.5f}".format( error2,corr)) 

Error: 0.65341 Correct: 0.70730
CPU times: user 18.4 s, sys: 10.9 s, total: 29.3 s
Wall time: 14.9 s


# Regularization  

A subset of methods used to encourage generalization in learned models, often by increasing the difficulty for a model to learn the fine-g rained details of training data.  

## 8.6  The Simplest Regularization: Early 

Stopping  Stop training the network when it starts getting worse.  

## Regularization: Dropout

Dropout makes our big n etwork act like a little one by randomly training  little subsections of the network at a time, and little networks don't overfit.  

Dropout is actually a form of training a bunch of networks and averaging them   

Randomly turn off a bunch of layers, reducing the neural network.

The takeaway is this; if you train 100 neural networks (all initialized randomly), they will each tend to latch onto different noise but similar broad  signal . Thus, when they mak e mistakes, they often make  differing  mistakes. This means that if we allowed them to vote equal y, their noise would tend to cancel out, revealing only what they have all learned in common,  the signal . 


In [ ]:
%%time
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# reshaping the matrix and normalizing the pixels colors (reminder than 0==zero and 255==white) dividing by 255
images, labels = (x_train[0:1000].reshape(1000,28*28)/255, y_train[0:1000])  

# create a matrix of zeros
one_hot_labels = np.zeros((len(labels),10))  

# make one_hot_label[i][l], where i(index of labels values) and l(labels values)), be 1 
for i,l in enumerate(labels):  
    one_hot_labels[i][l] = 1
    
labels = one_hot_labels  

# create the tests arrays
test_images = x_test.reshape(len(x_test),28*28) / 255  
test_labels = np.zeros((len(y_test),10))

# make test_labels[i][l], where i(index of y_test values) and l(y_test values)), be 1 
for i,l in enumerate( y_test):  
    test_labels[i][l] = 1

np.random.seed(1)  

# exclude negative numbers
def relu(x):  
  return (x >= 0) * x # returns x if x > 0 

# exclude negative numbers in derivative
def relu2deriv(output):      
  return output >= 0 #returns 1 for input > 0    
  
# learn rate, iterations, num of columns or rows of weights  
alpha, iterations, hidden_size = (0.005, 300, 100)  
pixels_per_image, num_labels = (784, 10)   

#ramdonly iniciate the weights
weights_0_1 = 0.2*np.random.random((pixels_per_image,hidden_size)) - 0.1  
weights_1_2 = 0.2*np.random.random((hidden_size,num_labels)) - 0.

error_list = []
score_list = []

for j in range(iterations):     
  error, correct_cnt = (0.0,0)     
  for i in range(len(images)): 
    #input image       
    layer_0 = images[i:i+1] 

    # make the weighted sum of layer_0(input) and remove the negative values       
    layer_1 = relu(np.dot(layer_0,weights_0_1))  

    # make the 0's and 1's matrix to multiple layer_1 and turn of the half of node
    # randint(max_number, size of matrix)    
    dropout_mask = np.random.randint(2, size=layer_1.shape) 

    # needs multiple to counter the turn off the half of layer_1 nodes     
    layer_1 *= dropout_mask * 2        

    # make the weighted sum of layer 1
    layer_2 = np.dot(layer_1,weights_1_2)  

    # calcule the square error       
    error += np.sum((labels[i:i+1] - layer_2) ** 2)  

     # counter the correct answer, comparing with the labels arg max (1)
    # reminder than layer and labels are arrays in format [0, 0, 1, 0] and [0.34, .65, .67, 1], so picking the argmax is pick the index of 1, and compare     
    correct_cnt += int(np.argmax(layer_2) == np.argmax(labels[i:i+1]))    

    # make the derivative     
    layer_2_delta = (labels[i:i+1] - layer_2)        
    layer_1_delta = layer_2_delta.dot(weights_1_2.T) * relu2deriv(layer_1) 

    # turn off the half of node in layer_1       
    layer_1_delta *= dropout_mask        

    # apply the gradient descendent in weights
    weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)        
    weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)   
  # making the tests  
  if (j%10 == 0):     
    test_error = 0.0     
    test_correct_cnt = 0      
    for i in range(len(test_images)):   
      #input image test  
      layer_0 = test_images[i:i+1]   

      #np.dot and relu of layer_1 test     
      layer_1 = relu(np.dot(layer_0,weights_0_1))        

      #np.dot and relu of layer_2 test  
      layer_2 = np.dot(layer_1, weights_1_2)  

      # calculate the square error       
      test_error += np.sum((test_labels[i:i+1] - layer_2) ** 2)     

      
      # counter the correct answer, comparing with the labels arg max (1)
      # reminder than layer and labels are arrays in format [0, 0, 1, 0] and [0.34, .65, .67, 1], so picking the argmax is pick the index of 1, and compare 
      test_correct_cnt += int(np.argmax(layer_2) ==  np.argmax(test_labels[i:i+1]))  
      
      # calculate the mean error
      error2 = test_error/ float(len(test_images))

      # calculate the % of correct answer
      correct2 = test_correct_cnt/ float(len(test_images))
      
      error_list.append(error2)
      score_list.append(correct2)
      
      # print the results
       
    print("I:" + str(j)+ \
          " Test- Err:" + str(test_error/ float(len(test_images)))[0:5] + \
          " Test- Acc:" + str(test_correct_cnt/ float(len(test_images)))+ \
          " Train- Err:" + str(error/ float(len(images)))[0:5] + \
          " Train-ACC:" + str(correct_cnt/ float(len(images)))[0:5])



I:0 Test- Err:0.831 Test- Acc:0.5087 Train- Err:1.021 Train-ACC:0.239
I:10 Test- Err:0.535 Test- Acc:0.7446 Train- Err:0.556 Train-ACC:0.732
I:20 Test- Err:0.424 Test- Acc:0.8028 Train- Err:0.435 Train-ACC:0.8
I:30 Test- Err:0.409 Test- Acc:0.8173 Train- Err:0.413 Train-ACC:0.819
I:40 Test- Err:0.423 Test- Acc:0.8033 Train- Err:0.402 Train-ACC:0.856
I:50 Test- Err:0.403 Test- Acc:0.8083 Train- Err:0.402 Train-ACC:0.832
I:60 Test- Err:0.415 Test- Acc:0.8003 Train- Err:0.396 Train-ACC:0.852
I:70 Test- Err:0.408 Test- Acc:0.809 Train- Err:0.374 Train-ACC:0.853
I:80 Test- Err:0.398 Test- Acc:0.8143 Train- Err:0.378 Train-ACC:0.865
I:90 Test- Err:0.404 Test- Acc:0.8077 Train- Err:0.368 Train-ACC:0.87
I:100 Test- Err:0.405 Test- Acc:0.8024 Train- Err:0.367 Train-ACC:0.87
I:110 Test- Err:0.407 Test- Acc:0.8128 Train- Err:0.360 Train-ACC:0.871
I:120 Test- Err:0.392 Test- Acc:0.814 Train- Err:0.364 Train-ACC:0.867
I:130 Test- Err:0.404 Test- Acc:0.8064 Train- Err:0.348 Train-ACC:0.855
I:140 Tes

## 8.11  Batch Gradient Descent

In [ ]:

%%time
np.random.seed(1)  

# here stay relu, but her was defined ago, so i just use her again

# same here to relu2derive

batch_size = 100  
alpha, iterations = (0.001, 300)  
pixels_per_image, num_labels, hidden_size = (784, 10, 100)  
weights_0_1 = 0.2*np.random.random((pixels_per_image,hidden_size)) - 0.1  
weights_1_2 = 0.2*np.random.random((hidden_size,num_labels)) - 0.1  
for j in range(iterations):  
  error, correct_cnt = (0.0, 0)  
  for i in range(int(len(images) / batch_size)):  

    # pick the indexes to slice the dataset
    batch_start, batch_end = ((i * batch_size),((i+1)*batch_size))  
    
    # slice the dataset
    layer_0 = images[batch_start:batch_end]  

    layer_1 = relu(np.dot(layer_0,weights_0_1)) 

    # create the mask to turno the layers
    dropout_mask = np.random.randint(2,size=layer_1.shape)  
    layer_1 *= dropout_mask * 2  
    layer_2 = np.dot(layer_1,weights_1_2)  
    
    error += np.sum((labels[batch_start:batch_end] - layer_2) ** 2) 
    
    # to smooth the weight upgrade, making this with the mean
    for k in range(batch_size):  
      
      # counter the correct answer, comparing with the labels arg max (1)
      # reminder than layer and labels are arrays in format [0, 0, 1, 0] and [0.34, .65, .67, 1], so picking the argmax is pick the index of 1, and compare
      correct_cnt += int(np.argmax(layer_2[k:k+1]) == np.argmax(labels[batch_start+k:batch_start+k+1]))  

      layer_2_delta = (labels[batch_start:batch_end]- layer_2)/batch_size  
      layer_1_delta = layer_2_delta.dot(weights_1_2.T)*relu2deriv(layer_1)  
      layer_1_delta *= dropout_mask  

      weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)  
      weights_0_1 += alpha * layer_0.T.dot(layer_1_delta) 

  # making the tests  
  if (j%10 == 0):     
    test_error = 0.0     
    test_correct_cnt = 0      
    for i in range(len(test_images)):   
      #input image test  
      layer_0 = test_images[i:i+1]   

      #np.dot and relu of layer_1 test     
      layer_1 = relu(np.dot(layer_0,weights_0_1))        

      #np.dot and relu of layer_2 test  
      layer_2 = np.dot(layer_1, weights_1_2)  

      # calculate the square error       
      test_error += np.sum((test_labels[i:i+1] - layer_2) ** 2)     

      
      # calculate the rights predictions, picking the biggest value of layer_2, remember than layer_2 is the probabilities of be one value 
      # and comparing with the correct answer(label)  
      test_correct_cnt += int(np.argmax(layer_2) ==  np.argmax(test_labels[i:i+1]))  
    
    # print the results  
    print("I:" + str(j)+ \
          " Test- Err:" + str(test_error/ float(len(test_images)))[0:5] + \
          " Test- Acc:" + str(test_correct_cnt/ float(len(test_images)))+ \
          " Train- Err:" + str(error/ float(len(images)))[0:5] + \
          " Train-ACC:" + str(correct_cnt/ float(len(images)))[0:5])



I:0 Test- Err:0.815 Test- Acc:0.3832 Train- Err:1.284 Train-ACC:0.165
I:10 Test- Err:0.568 Test- Acc:0.7173 Train- Err:0.591 Train-ACC:0.672
I:20 Test- Err:0.510 Test- Acc:0.7571 Train- Err:0.532 Train-ACC:0.729
I:30 Test- Err:0.485 Test- Acc:0.7793 Train- Err:0.498 Train-ACC:0.754
I:40 Test- Err:0.468 Test- Acc:0.7877 Train- Err:0.489 Train-ACC:0.749
I:50 Test- Err:0.458 Test- Acc:0.793 Train- Err:0.468 Train-ACC:0.775
I:60 Test- Err:0.452 Test- Acc:0.7995 Train- Err:0.452 Train-ACC:0.799
I:70 Test- Err:0.446 Test- Acc:0.803 Train- Err:0.453 Train-ACC:0.792
I:80 Test- Err:0.451 Test- Acc:0.7968 Train- Err:0.457 Train-ACC:0.786
I:90 Test- Err:0.447 Test- Acc:0.795 Train- Err:0.454 Train-ACC:0.799
I:100 Test- Err:0.448 Test- Acc:0.793 Train- Err:0.447 Train-ACC:0.796
I:110 Test- Err:0.441 Test- Acc:0.7943 Train- Err:0.426 Train-ACC:0.816
I:120 Test- Err:0.442 Test- Acc:0.7966 Train- Err:0.431 Train-ACC:0.813
I:130 Test- Err:0.441 Test- Acc:0.7906 Train- Err:0.434 Train-ACC:0.816
I:140 T